In [ ]:
from qiskit.circuit.quantumregister import QuantumRegister
from qiskit.dagcircuit.dagcircuit import DAGCircuit
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.tools.visualization import dag_drawer
import supremacy_generator as suprem_gen
from qiskit import *
from qiskit.visualization import plot_state_city
import numpy as np
import functools
import timeit
from datetime import timedelta
import supremacy_generator as suprem_gen
import matplotlib.pyplot as plt
from matplotlib import cm

dimensions = range(1,6)
num_qubits = [x*x for x in dimensions]
sv_times = np.zeros((len(dimensions), len(dimensions)))
qasm_times = np.zeros((len(dimensions), len(dimensions)))
difference = np.zeros((len(dimensions), len(dimensions)))

for i in dimensions:
    for j in dimensions:
        if i*j<=24:
            circ = suprem_gen.circuit_generator(circuit_dimension=[i,j,8])

            sv_start = timeit.default_timer()
            backend = BasicAer.get_backend('statevector_simulator')
            job = execute(circ, backend)
            result = job.result()
            outputstate = result.get_statevector(circ)
            sv_outputprob = [np.power(abs(x),2) for x in outputstate]
            sv_end = timeit.default_timer()
            sv_times[i-1,j-1] = sv_end-sv_start

            qasm_start = timeit.default_timer()
            num_measurements = int(np.power(2,i*j))
            c = ClassicalRegister(i*j, 'c')
            meas = QuantumCircuit(circ.qregs[0], c)
            meas.barrier(circ.qubits)
            meas.measure(circ.qubits,c)
            qc = circ+meas

            backend_sim = BasicAer.get_backend('qasm_simulator')
            job_sim = execute(qc, backend_sim, shots=num_measurements)
            result_sim = job_sim.result()
            counts = result_sim.get_counts(qc)
            qasm_outputprob = [counts[x]/num_measurements for x in counts]
            qasm_end = timeit.default_timer()
            qasm_times[i-1,j-1] = qasm_end-qasm_start
            difference[i-1,j-1] = sv_times[i-1,j-1] - qasm_times[i-1,j-1]
        else:
            sv_times[i-1,j-1] = 0
            qasm_times[i-1,j-1] = 0
            difference[i-1,j-1] = sv_times[i-1,j-1] - qasm_times[i-1,j-1]

print('sv_times:\n', sv_times)
print('qasm_times:\n', qasm_times)
print('sv - qasm:\n', difference)

plt.figure(figsize=(10,10))
plt.subplot(221)
plt.imshow(sv_times, aspect='auto',
           extent=[min(dimensions), max(dimensions), min(dimensions), max(dimensions)], cmap = cm.viridis)
plt.xlabel('#horizontal qubits')
plt.ylabel('#vertical qubits')
plt.title('state vector simulation time')
plt.colorbar()
plt.subplot(222)
plt.imshow(qasm_times, aspect='auto',
           extent=[min(dimensions), max(dimensions), min(dimensions), max(dimensions)], cmap = cm.viridis)
plt.colorbar()
plt.xlabel('#horizontal qubits')
plt.title('qasm simulation time')
plt.subplot(223)
plt.imshow(difference, aspect='auto',
           extent=[min(dimensions), max(dimensions), min(dimensions), max(dimensions)], cmap = cm.viridis)
plt.colorbar()
plt.title('sv-qasm time difference')
plt.xlabel('#horizontal qubits')
plt.ylabel('#vertical qubits')
plt.show()